# Manual ingest workflow, bypassing GUI

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging, behavior
from adamacs.ingest import session as isess
from adamacs.helpers import stack_helpers as sh
from adamacs.ingest import behavior as ibe
from natsort import natsorted, ns
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-11-06 16:31:38,849][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-11-06 16:31:38,895][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.1'

### RSpace connection

In [2]:
URL=dj.config['custom'].get('rspace_URL')
API_KEY=dj.config['custom'].get('rspace_API_key')
api = eln.ELNClient(URL, API_KEY)
api.get_status()

{'message': 'OK', 'rspaceVersion': '1.89.2'}

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [3]:
from adamacs.utility import *
# from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment

Assign easy names for relevant tables

In [4]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

some helpers

In [5]:

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_subject_key_from_dir(string):
    result = [item.split("_")[1] for item in string]
    return result

def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

## Parse directories

In [6]:
# get content of user directory
import fnmatch
dataroot = dj.config['custom']['exp_root_data_dir'][0]
# dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and '_' in d]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and fnmatch.fnmatch(d, 'JJ*') and fnmatch.fnmatch(d, '*1-*')]
sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)
sorted_dirs_root

['JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3',
 'JJ_ROS_1438_2022-11-22_scan9FF6TL96_sess9FF6TL96']

In [7]:
sessi = "sess9FKW82R3"
scansi = "scan9FKW82R3"

## Session ingest

In [21]:
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')
s2pparm = imaging.ProcessingParamSet.fetch("paramset_idx", "paramset_desc")

print(project)
print(Equipment)
print(Recording_Location)
print(s2pparm)

*project       project_descri
+------------+ +------------+
ATN            ATN-functional
dummy          dummy         
hpc-repstab    hpc-representa
rsc-functop    rsc-functional
rsc-hpc        rsc-hippocampa
rsc-latent     rsc-contextual
sc-lgn-actvis  sc-lgn-active-
V1-oddball     v1-oddball-pre
vc-lgn-repstab vc-lgn-represe
 (Total: 9)

['bench2p' 'dummy' 'macroscope' 'mini2p_01' 'mini2p_02' 'mini2p_03'
 'mini2p_04' 'mini2p_05']
['ATN' 'Ctx' 'dCA1' 'DG' 'dummy' 'LGNV1' 'Location' 'RSCa' 'RSCg' 'V1']
[array([0, 1, 2, 3, 4, 5, 6, 7]), array(['TR: Mini2p (new, non-rigid, individual scans, reg_tiff for movie, custom classifier)',
       'TR: Mini2p (rigid, mini2p classifier, no concat, REGTIFF)',
       'TR: Trondheim Mini2p (non-rigid, built-in classifier, scan_concat)',
       'TR: Bench2p (non-rigid, custom classifier, reg_tiff for movie, scans individually)',
       'TR: Bench2p (rigid, custom bench2p classifier, individual, SAVETIF)',
       'TR: Mini2p (rigid, built-in classifier

Ingest Session and Scan

In [36]:
isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
session.SessionSameSite.update1({'session_id': sessi, 'same_site_id': sessi})


Number of scans found: 1
[['scan9FKW82R3']]
['JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3']


/tmp/ipykernel_2953805/4141465353.py:1: UserWarning: 
Skipped existing session row: ('sess9FKW82R3', <module 'adamacs.schemas.subject' from '/home/tobiasr/adamacs/adamacs/schemas/subject.py'>, '2023-11-02')
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_2953805/4141465353.py:1: UserWarning: 
Skipped existing ProjectSession row: ('rsc-functop', 'sess9FKW82R3')
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_2953805/4141465353.py:1: UserWarning: 
Skipped existing SessionDirectory: ('sess9FKW82R3', PosixPath('/datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3'))
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_2953805

In [16]:
scan.Scan * session.SessionSameSite * session.Session() & f'session_id = "{sessi}"'

session_id,scan_id,scanner,acq_software,scan_notes free-notes,same_site_id,subject PyRat import uses this for earmark value,session_datetime
sess9FKW82R3,scan9FKW82R3,mini2p_01,ScanImage,,sess9FKW82R3,ROS-1604,2023-11-02 00:00:00


In [23]:
query = scan.ScanPath() & 'scan_id = "' + scansi + '"'
dir_proc = query.fetch('path')[0]
print(dir_proc)

/datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3


In [24]:
# push scan to ProcessingTask
# TODO: handle multiscan concatenation from here?
selected_s2pparms_index = 0
imaging.ProcessingTask.insert1((sessi, scansi, selected_s2pparms_index, dir_proc, 'trigger'), skip_duplicates=True)

In [39]:
# POPULATE!
populate_settings = {'display_progress': True, 'suppress_errors': False, 'processes': 1}
scan.ScanInfo.populate(**populate_settings) 

ScanInfo: 100%|██████████| 1/1 [00:00<00:00,  9.85it/s]

Multiple .tif files found. Chose /datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3/scan9FKW82R3_JJ-ROS-1604_00002.tif for scanimage header


## Ingest AUX

In [41]:
aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]    
print(aux_setup_typestr)

mini2p1_openfield


In [42]:
ibe.ingest_aux(sessi,scansi,verbose=True, aux_setup_type=aux_setup_typestr)

Number of aux-files found: 1
[PosixPath('/datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3')]


/home/tobiasr/miniconda3/envs/datajoint-DLCbackup/lib/python3.8/site-packages/pywavesurfer/ws.py:46: RuntimeWarning: You are reading a WaveSurfer file version this module was not tested with: file version 1.0.6, latest version tested: 0.982
  warnings.warn('You are reading a WaveSurfer file version this module was not tested with: '


UnboundLocalError: local variable 'event_types' referenced before assignment

## Cleaning. Use with caution!

In [ ]:
# subject.Subject.delete()
# session.Session.delete()
# imaging.Processing.delete()
# imaging.Curation.delete()
# event.Event.delete()
# event.BehaviorRecording.delete()

In [38]:
(session.Session & "session_id LIKE 'sess9FKOI7ZY%'").delete()
(imaging.ProcessingTask & "session_id LIKE 'sess9FKOI7ZY%'").delete()
# (subject.Subject & "subject = 'ROS-1571'").delete()
# # subject.Subject.delete()
# # session.Session.delete()
# # imaging.Processing.delete()
# # imaging.Curation.delete()
# # event.Event.delete()
# # event.BehaviorRecording.delete()

[2023-11-06 16:52:17,110][INFO]: Deleting 1 rows from `tobiasr_imaging`.`processing_task`
[2023-11-06 16:52:17,114][INFO]: Deleting 1 rows from `tobiasr_scan`.`scan_location`
[2023-11-06 16:52:17,118][INFO]: Deleting 1 rows from `tobiasr_scan`.`scan_path`
[2023-11-06 16:52:17,121][INFO]: Deleting 1 rows from `tobiasr_scan`.`scan`
[2023-11-06 16:52:17,124][INFO]: Deleting 1 rows from `tobiasr_session`.`project_session`
[2023-11-06 16:52:17,128][INFO]: Deleting 1 rows from `tobiasr_session`.`session_directory`
[2023-11-06 16:52:17,132][INFO]: Deleting 1 rows from `tobiasr_session`.`session_same_site`
[2023-11-06 16:52:17,136][INFO]: Deleting 1 rows from `tobiasr_session`.`session_user`
[2023-11-06 16:52:17,138][INFO]: Deleting 1 rows from `tobiasr_session`.`session`
[2023-11-06 16:52:18,950][INFO]: Deletes committed.
[2023-11-06 16:52:18,953][INFO]: Deleting 0 rows from `tobiasr_imaging`.`processing_task`
[2023-11-06 16:52:18,953][WARNING]: Nothing to delete.


0

In [ ]:
# session.Session.drop()
# scan.Scan.drop()
# imaging.Processing.drop()
# imaging.Curation.drop()